In [1]:
import os

In [2]:
%pwd

'c:\\Users\\mdhoz\\OneDrive\\Desktop\\Coding\\Machine_Learning\\Kidney-Disease-Classificatioin-DeepLearning\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\mdhoz\\OneDrive\\Desktop\\Coding\\Machine_Learning\\Kidney-Disease-Classificatioin-DeepLearning'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training_config = self.config.training
        prepare_base_model = self.config.prepare_base_model
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        training_params = self.params

        create_directories([Path(training_config.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training_config.root_dir),
            trained_model_path=Path(training_config.trained_model_path),
            updated_base_model_path=Path(
                prepare_base_model.updated_base_model_path
            ),
            training_data=Path(training_data),
            params_epochs=training_params.EPOCHS,
            params_batch_size=training_params.BATCH_SIZE,
            params_is_augmentation=training_params.AUGMENTATION,
            params_image_size=training_params.IMAGE_SIZE,
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
tf.config.run_functions_eagerly(True)  # For debugging purposes

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        # Load model without compiling to avoid restoring optimizer state
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path,
            compile=False
        )

        # Re-compile model appropriately depending on final output shape
        output_shape = self.model.output_shape
        # If final dim == 1 -> binary; else multi-class
        if len(output_shape) >= 2 and output_shape[-1] == 1:
            loss = "binary_crossentropy"
            optimizer = tf.keras.optimizers.Adam()
        else:
            loss = "sparse_categorical_crossentropy"
            optimizer = tf.keras.optimizers.Adam()

        self.model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

    def train_valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.20,
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        validation_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # produce integer labels compatible with sparse_categorical_crossentropy
        self.validation_generator = validation_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            class_mode="sparse",
            **dataflow_kwargs,
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=20,
                horizontal_flip=True,
                vertical_flip=True,
                width_shift_range=0.1,
                height_shift_range=0.1,
                shear_range=0.1,
                zoom_range=0.2,
                **datagenerator_kwargs,
            )
        else:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                **datagenerator_kwargs
            )

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            class_mode="sparse",
            **dataflow_kwargs,
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.validation_generator.samples // self.validation_generator.batch_size

        self.model.fit(
            self.train_generator,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.validation_generator,
            validation_steps=self.validation_steps,
            epochs=self.config.params_epochs
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()

except Exception as e:
    raise e

[2025-10-27 22:42:27,134: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-27 22:42:27,143: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-27 22:42:27,146: INFO: common: Directory: artifacts created successfully]
[2025-10-27 22:42:27,148: INFO: common: Directory: artifacts\training created successfully]
[2025-10-27 22:42:27,143: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-27 22:42:27,146: INFO: common: Directory: artifacts created successfully]
[2025-10-27 22:42:27,148: INFO: common: Directory: artifacts\training created successfully]
Found 93 images belonging to 2 classes.
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Found 372 images belonging to 2 classes.


c:\Users\mdhoz\miniconda3\envs\kidney\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\mdhoz\miniconda3\envs\kidney\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
c:\Users\mdhoz\miniconda3\envs\kidney\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data fu

23/23 ━━━━━━━━━━━━━━━━━━━━ 189s 8s/step - accuracy: 0.7908 - loss: 0.7338 - val_accuracy: 1.0000 - val_loss: 0.0492
23/23 ━━━━━━━━━━━━━━━━━━━━ 189s 8s/step - accuracy: 0.7908 - loss: 0.7338 - val_accuracy: 1.0000 - val_loss: 0.0492
[2025-10-27 22:45:36,972: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
[2025-10-27 22:45:36,972: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
